<a href="https://colab.research.google.com/github/SheikShaheda-6904/Efficient-Leftover-Food-Management-System/blob/main/Efficient_Leftover_Food_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from IPython.display import display, HTML


def initialize_system():

    try:
        inventory_df = pd.read_csv('inventory.csv', index_col='Item_ID')
        print("Inventory loaded from inventory.csv.")
    except FileNotFoundError:
        inventory_data = {
            'Item_ID': [101, 102, 103],
            'Name': ['Apples', 'Milk', 'Bread'],
            'Quantity': [100, 50, 75],
            'Unit_Cost': [0.50, 1.25, 2.00],
            'Selling_Price': [0.75, 1.80, 2.50]
        }
        inventory_df = pd.DataFrame(inventory_data).set_index('Item_ID')
        print("Default Inventory created.")


    try:
        sales_df = pd.read_csv('sales.csv')
        print("Sales log loaded from sales.csv.")
    except FileNotFoundError:
        sales_df = pd.DataFrame(columns=['Sale_ID', 'Item_ID', 'Quantity_Sold', 'Total_Price', 'Timestamp'])
        sales_df['Timestamp'] = pd.to_datetime(sales_df['Timestamp'])
        print("New Sales log created.")

    return inventory_df, sales_df

inventory_df, sales_df = initialize_system()


Default Inventory created.
New Sales log created.


In [14]:


def add_new_item(name, quantity, unit_cost, selling_price):
    """Adds a new item to the inventory."""
    global inventory_df

    new_id = inventory_df.index.max() + 1 if not inventory_df.empty else 101

    new_item = pd.DataFrame({
        'Name': [name],
        'Quantity': [quantity],
        'Unit_Cost': [unit_cost],
        'Selling_Price': [selling_price]
    }, index=[new_id])
    new_item.index.name = 'Item_ID'

    inventory_df = pd.concat([inventory_df, new_item])
    print(f"✅ Item '{name}' added with ID {new_id}.")

def view_inventory():
    """Displays the current inventory with calculated value."""

    inventory_df['Total_Cost_Value'] = inventory_df['Quantity'] * inventory_df['Unit_Cost']
    inventory_df['Total_Sale_Value'] = inventory_df['Quantity'] * inventory_df['Selling_Price']

    print("\n--- 🛒 Current Inventory ---")
    display(HTML(inventory_df.to_html()))
    print(f"Total Inventory Cost Value: ${inventory_df['Total_Cost_Value'].sum():.2f}")
    inventory_df.drop(columns=['Total_Cost_Value', 'Total_Sale_Value'], inplace=True) # Clean up temp columns

def record_sale(item_id, quantity_sold):
    """Records a sale and updates the inventory stock."""
    global inventory_df, sales_df

    if item_id not in inventory_df.index:
        print(f"❌ Error: Item ID {item_id} not found.")
        return

    item = inventory_df.loc[item_id]
    item_name = item['Name']

    if quantity_sold > item['Quantity']:
        print(f"❌ Error: Not enough stock for {item_name}. Available: {item['Quantity']}.")
        return


    inventory_df.loc[item_id, 'Quantity'] -= quantity_sold


    new_sale_id = sales_df['Sale_ID'].max() + 1 if not sales_df.empty else 1
    new_sale = pd.DataFrame({
        'Sale_ID': [new_sale_id],
        'Item_ID': [item_id],
        'Quantity_Sold': [quantity_sold],
        'Total_Price': [quantity_sold * item['Selling_Price']],
        'Timestamp': [pd.Timestamp.now()]
    })
    sales_df = pd.concat([sales_df, new_sale], ignore_index=True)

    print(f"✅ Sale recorded: {quantity_sold} of {item_name} for ${quantity_sold * item['Selling_Price']:.2f}.")

def generate_sales_report():
    """Generates a summary report of all sales, profit, and revenue."""
    if sales_df.empty:
        print("No sales recorded yet.")
        return

    print("\n--- 📈 Sales Report ---")


    report_df = sales_df.merge(inventory_df[['Name', 'Unit_Cost', 'Selling_Price']],
                                left_on='Item_ID', right_index=True, how='left')


    report_df['COGS'] = report_df['Quantity_Sold'] * report_df['Unit_Cost']
    report_df['Profit'] = report_df['Total_Price'] - report_df['COGS']


    total_revenue = report_df['Total_Price'].sum()
    total_cogs = report_df['COGS'].sum()
    total_profit = report_df['Profit'].sum()

    print(f"Total Revenue: ${total_revenue:.2f}")
    print(f"Total COGS:    ${total_cogs:.2f}")
    print(f"**Total Profit:** ${total_profit:.2f}")


    item_summary = report_df.groupby('Name').agg(
        Total_Units_Sold=('Quantity_Sold', 'sum'),
        Total_Revenue=('Total_Price', 'sum'),
        Total_Profit=('Profit', 'sum')
    ).sort_values(by='Total_Profit', ascending=False)

    print("\nSales Breakdown by Item:")
    display(HTML(item_summary.to_html()))

In [15]:
view_inventory()


--- 🛒 Current Inventory ---


,Name,Quantity,Unit_Cost,Selling_Price,Total_Cost_Value,Total_Sale_Value
Item_ID,,,,,,
101,Apples,100,0.50,0.75,50.0,75.0
102,Milk,50,1.25,1.80,62.5,90.0
103,Bread,75,2.00,2.50,150.0,187.5


Total Inventory Cost Value: $262.50


In [16]:
add_new_item(name="Oranges", quantity=200, unit_cost=0.30, selling_price=0.50)
view_inventory()

✅ Item 'Oranges' added with ID 104.

--- 🛒 Current Inventory ---


,Name,Quantity,Unit_Cost,Selling_Price,Total_Cost_Value,Total_Sale_Value
Item_ID,,,,,,
101,Apples,100,0.50,0.75,50.0,75.0
102,Milk,50,1.25,1.80,62.5,90.0
103,Bread,75,2.00,2.50,150.0,187.5
104,Oranges,200,0.30,0.50,60.0,100.0


Total Inventory Cost Value: $322.50


In [17]:

record_sale(item_id=101, quantity_sold=10)


record_sale(item_id=102, quantity_sold=5)


record_sale(item_id=104, quantity_sold=50)

view_inventory()

✅ Sale recorded: 10 of Apples for $7.50.
✅ Sale recorded: 5 of Milk for $9.00.
✅ Sale recorded: 50 of Oranges for $25.00.

--- 🛒 Current Inventory ---


/tmp/ipython-input-3277746603.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sales_df = pd.concat([sales_df, new_sale], ignore_index=True)


,Name,Quantity,Unit_Cost,Selling_Price,Total_Cost_Value,Total_Sale_Value
Item_ID,,,,,,
101,Apples,90,0.50,0.75,45.00,67.5
102,Milk,45,1.25,1.80,56.25,81.0
103,Bread,75,2.00,2.50,150.00,187.5
104,Oranges,150,0.30,0.50,45.00,75.0


Total Inventory Cost Value: $296.25


In [18]:
generate_sales_report()


--- 📈 Sales Report ---
Total Revenue: $41.50
Total COGS:    $26.25
**Total Profit:** $15.25

Sales Breakdown by Item:


,Total_Units_Sold,Total_Revenue,Total_Profit
Name,,,
Oranges,50,25.0,10.0
Milk,5,9.0,2.75
Apples,10,7.5,2.5


In [19]:
from google.colab import drive

drive.mount('/content/drive')

def save_data_to_drive(inventory_df, sales_df, folder_path='/content/drive/My Drive/Colab_Food_System/'):
    """Saves the current DataFrames as CSV files to Google Drive."""

    import os


    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    inventory_path = os.path.join(folder_path, 'inventory.csv')
    sales_path = os.path.join(folder_path, 'sales.csv')

    inventory_df.to_csv(inventory_path, index=True)
    sales_df.to_csv(sales_path, index=False)

    print(f"✅ Data saved successfully to: {folder_path}")
    print("To load next time, change the `initialize_system` function to read from these paths.")


save_data_to_drive(inventory_df, sales_df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data saved successfully to: /content/drive/My Drive/Colab_Food_System/
To load next time, change the `initialize_system` function to read from these paths.
